In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, utils
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score, accuracy_score
from skimage.measure import EllipseModel, find_contours
from skimage.draw import ellipse
import nibabel as nib
import SimpleITK as sitk
import numpy as np
import seaborn as sns
from scipy import ndimage, spatial, special
import scipy.optimize as optimize
import os
from radiomics import featureextractor
import radiomics
import numpy as np
import pandas as pd
import math
import datatable as dt
from skimage import io, transform
import seaborn as sns
import pickle
from torch.utils.data import DataLoader, Dataset
import glob
from PIL import Image
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
try:
    num_gpus = torch.cuda.device_count()
    print("Number of GPUs available:", num_gpus)
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
except Exception as e:
    print("Error while checking GPUs:", e)

Number of GPUs available: 3
GPU 0: NVIDIA RTX A6000
GPU 1: NVIDIA RTX A6000
GPU 2: NVIDIA RTX A6000


In [3]:
print(torch.cuda.is_available())
device = torch.device("cuda:1")

True
